In [2]:
import pandas as pd

## Loading the parque files
as you can see the parque function recovers the field names. 

In [41]:
pageview = pd.read_parquet('./fct_page_view_nl.parque')
pageview.head()

,_surrogate_key,dateAmsterdam,eventTimestamp,eventId,pageName,hookId,visitId,isWebsite,isAndroid,isIOS
0,bc6c663d8987017154a859a3d60e00ba,2022-11-06,2022-11-06 16:25:29.898,h9K-7q4Rx4Ognl3IkFxNJw,RA,None,+B/A5vuJu0CzjZpdnTnjmUrEb8CgZQtg/ammnrmRk/0,1,0,0
1,67f4edd1c0997a3131e2e0b6fcb08c8d,2022-11-06,2022-11-06 16:12:48.301,o0yOzo7S0jS3cwPxhxhFnQ,RA,o9TyTRhReXmfnWCCoxVCEQ.4_10.11.Banner,/LTTUwXyKNTlJUUmy/6YXU5icZNy5mWUEXQbp+Hrh7E,1,0,0
2,82551fd0570e41e9903b63d74a90d85d,2022-11-06,2022-11-06 20:35:14.848,lXcQHWOKj-eqm5OumhtIQQ,RA,iQrm94vMJLtJowlVJHBPwQ.5_8.9.Banner,0l9HX7jbGdA/RgKc8bwfUMHjolcq6gj0s+/mffPhu6Y,1,0,0
3,d375b023dd4f79ebadd044d6a3055760,2022-11-06,2022-11-06 21:00:49.837,nPzVqpKKJ5xUTJEXlo5Faw,RA,iKfP-nFesQu7W90sMM5CTg.43_50.51.Banner,0tlyA2jn2G//z1iW2AXOhymd53nQlDK8DRJ11Q10lqA,1,0,0
4,55dd5a271c225756560d805a83714632,2022-11-06,2022-11-06 12:10:58.158,q5zymTbPTpiZ1dcb0k1PYg,RA,jah--Wmy4OQGorwng7NO1Q.4_10.11.Banner,1bG0AB3f4OW1g2fh/hseLfJALg2VLGWw6/8x4QbIVUE,1,0,0


## Structure of the data

### data subjects

This dataset is only including certain visitIds that are part of a so called A/A, test was performed to create a dataset that can be analyzed. The randomization took place on deviceId.

You can join the pageviews back to the `dim_ab_subjects_limited.parque` to find deviceIds or customerIds with multiple visitIds. This way, you are able to create a mapping of behavior and returning visits.

* a visitId is created on the first event, and expires after 30 minutes of inactivity with a max length of 12 hours. 
* a deviceId is an identifier consistent over long period of time (cookie expiry on this is set to the maximum of 12 years, but its likely cleared more often by browser clients).
* a customerId is only present when the user is logged in, the deviceId is always present.

In [65]:
subjects = pd.read_parquet('./dim_ab_subjects_limited.parque')

In [68]:
# for example
customerId = subjects['customerId']
subjects[customerId == b'\x05\xf2G\xef%z\x87{%\xa4\xa9-;\xb6\xf5\x99R\xb19P\x9f!;\xb1:\xc5N$\xc4\xf4\xc6\x16']

,dateAmsterdam,eventTimestamp,visitId,customerId,deviceId
48,2022-10-24,2022-10-24 16:17:51.290,3Se2rHF/ezhR0bF7DG7hhlSZ2uTAVNyC29UT/Sy9ZQ4,b'\x05\xf2G\xef%z\x87{%\xa4\xa9-;\xb6\xf5\x99R...,b'>}\xbbT5A\xc6E\xccmlB\xb1\x15\xee\xc9{\xf5#\...
140263,2022-11-07,2022-11-07 21:26:13.842,szhUCvRffBM23R45SXfsL2lRXz7a/+vmZK3osjlY0rw,b'\x05\xf2G\xef%z\x87{%\xa4\xa9-;\xb6\xf5\x99R...,b'DC\x03=\tV\x01n\xa8v\xc0}G\x00\x92\xacS\x8b?...


## Anatomy of a page view

## pageName

Each page view has a unique identifier the `eventId`. By using the `vistiId` and `eventTimestamp` you can sequence the shopper's customer journey over one visit or multiple visits by joining back to `dim_ab_subjects_limited.parque` file.

By looking at the pageName you'll be able to see if the visit converted to the wishlist or shopping basket and payment.


In [47]:
pageview.groupby(by='pageName')['eventId'].describe()

,count,unique,top,freq
pageName,,,,
AddonPage,11781,11781,qxMRgMBh-8HcQ6jwV8pOBQ,1
Basket,1397,1397,vLguQ-YOH0821yYgSg9MqQ,1
CMS,2647,2647,pR8hnc36seyAuIrztflNXg,1
Checkout,9874,9874,vlxIovL3y492TX3cyt9LAg,1
Home,27712,27712,gRRUwV2CtAqfderALC9ONw,1
OHP,41860,41860,nBYW6BnFPS-IBZQAVopO2g,1
PDP,460119,460119,nf0HquCg5lb9OqWrKjFHsA,1
PaymentDone,6679,6679,guCLxg5loyAjn2x-zg5Mbg,1
PriceComparePage,60542,60542,oSS23enL5xyIt58uOyhHyQ,1


### hookId

A hookId is stored on every element that is tagged to the item level (can be a banner, a review or a product, can be even a listpage filter or menu item).

You can use take the first part of a hookId (everything until the first dot charachter (`.`) which is the same as the eventId.

For example hookId = `o9TyTRhReXmfnWCCoxVCEQ.4_10.11.Banner`, which relates to eventId = `o9TyTRhReXmfnWCCoxVCEQ`. Thus you can perform reverse lookups to see which page the click originated from.
This might also become relevant when analyzing other datasets that give describe on product or banner impressions (item level).

In the hookId example `o9TyTRhReXmfnWCCoxVCEQ.4_10.11` is the itemId and "Banner" is the call to action from that UI item. Some items can have multiple call to actions, for example on a product there can be more than 5  "call to actions" or hookIds:

- product title
- product image
- add to basket
- add to wishlist
- compare with other product


## Show the number of visitors per day


In [26]:
pageview.groupby(by='dateAmsterdam')['visitId'].describe()

,count,unique,top,freq
dateAmsterdam,,,,
2022-10-24,35874,7496,nnFcxr7KJsFupOp+vdoLKeGiLv24QjZ/OOIUBes+jBo,589
2022-10-25,33069,7979,NELSJnR2LaXgNhKFlbJKeP41IUxSlu25KOv0C6xHIYI,237
2022-10-26,56791,7855,77FkBKzAlaGnrdJkPr0N5umYJqCwTRDnlMDmAb1Qa+M,19217
2022-10-27,32917,7869,ZSaOyxyOInyRVN6nTLgF/wFbJkyYYFJiQUE/EIuZMLs,212
2022-10-28,34831,7757,bVkoYa0l4W1JlOIKTqsfwxBrb7Ib+cxa/EAduVD1iLU,1109
2022-10-29,32935,7912,ngbsSz8b5YEWBWPTJnAUUUinbgRHHy3JLnP/d0U7VfY,172
2022-10-30,36410,8283,JBxzIoUK1zGq8GZVFgqb4O1eyAQh3ENZ4ys2lczYniU,284
2022-10-31,41750,7914,y0UvNswZYyPS2Gh4C/MLjUgy0Y4OosX1Z7UFwO/vkks,3855
2022-11-01,46110,7812,Xxo80gD5ChpGIR2QzUV1zKYWP9GAtAUCbXTzLUPwaVg,9224


## Convert data types example

by first creating a new dataframe where the data types are casted correctly, you can then use any available python function on the string, date int fields etc.

In [34]:
pageview = pageview.convert_dtypes()
pageview['dateAmsterdam'] = pd.to_datetime(pageview['dateAmsterdam'], format="%Y-%m-%d")
pageview.dtypes

_surrogate_key    string[python]
dateAmsterdam     datetime64[ns]
eventTimestamp    datetime64[ns]
eventId           string[python]
pageName          string[python]
hookId            string[python]
visitId           string[python]
isWebsite                  Int64
isAndroid                  Int64
isIOS                      Int64
dtype: object

In [33]:
pageview.eventTimestamp.map(lambda x: x.hour)

0         16
1         16
2         20
3         21
4         12
          ..
692749    11
692750     9
692751     7
692752     8
692753    16
Name: eventTimestamp, Length: 692754, dtype: int64